# 1. Validation Set Approach

In [14]:
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# load Boston dataset
boston = fetch_openml(name="Boston", version=1, as_frame=True)
df = boston.frame
df.columns = [c.lower() for c in df.columns]  # normalize column names to lowercase

# list of columns in the dataset
print("Columns in the Boston dataset:")
print(df.columns.tolist())

# predictors and response
X = df[['lstat']].values
y = df['medv'].values

# split into training and validation sets with exactly 253 observations each
X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size=253, test_size=253, random_state=42)

# fit polynomial models of degree 1-4 and compute validation MSEs
mses = []
for deg in range(1, 5):
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    X_train_p = poly.fit_transform(X_train)
    X_val_p = poly.transform(X_val)

    model = LinearRegression()
    model.fit(X_train_p, y_train)
    y_pred = model.predict(X_val_p)

    mses.append(mean_squared_error(y_val, y_pred))

# report results as a numpy array with 4 elements rounded to 2 decimals
print("Validation MSEs for polynomial degrees 1 to 4:")
mses = np.round(np.array(mses), 2)
print(mses)

Columns in the Boston dataset:
['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv']
Validation MSEs for polynomial degrees 1 to 4:
[38.51 30.84 29.22 27.75]


# 2. Leave-One-Out





In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

# load dataset
local_path = '/Users/anhnguyen/Documents/GitHub/2025AAE722_AnhNguyen/2025 AAE722 Anh Nguyen Submission/College.csv'
college = pd.read_csv(local_path)

# extract variables
X = college[['Room.Board']]
y = college['Outstate']

# set up LOOCV
n = len(college)  # number of observations
cv = n            # LOOCV: each observation is its own fold
cv_errors = []

for degree in range(1, 6):
    # create polynomial model pipeline
    model = make_pipeline(PolynomialFeatures(degree=degree, include_bias=False),
                          LinearRegression())
    
    # perform LOOCV using cross_validate
    results = cross_validate(model, X, y,
                             cv=cv,
                             scoring='neg_mean_squared_error',
                             n_jobs=-1)
    
    # compute mean cross-validation error (MSE)
    mean_mse = -np.mean(results['test_score'])
    cv_errors.append(mean_mse)

# report results as a numpy array with 5 elements rounded to 2 decimals
print("LOOCV MSEs for polynomial degrees 1 to 5:")
cv_errors = np.round(np.array(cv_errors), 2)
print(cv_errors)

LOOCV MSEs for polynomial degrees 1 to 5:
[9291471.1  9255509.68 9263314.52 9268010.36 9284911.37]


# 3. K-Fold


In [16]:
# import dataset
import os
local_path = '/Users/anhnguyen/Documents/GitHub/2025AAE722_AnhNguyen/2025 AAE722 Anh Nguyen Submission/College.csv'

if not os.path.exists(local_path):
    raise FileNotFoundError(f"File not found: {local_path}")

# read CSV (use index_col=0 as in the notebook)
college = pd.read_csv(local_path, index_col=0)

# quick checks
print("Loaded College.csv:", college.shape)
print("Columns:", college.columns.tolist())

from sklearn.model_selection import KFold

# set up KFold configurations
kf5 = KFold(n_splits=5, shuffle=True, random_state=123)
kf10 = KFold(n_splits=10, shuffle=True, random_state=123)

degrees = [1, 2, 3]
results = {}

for deg in degrees:
    pipe = make_pipeline(PolynomialFeatures(degree=deg, include_bias=False),
                         SMWrapper())
    # 5-fold CV
    scores5 = cross_validate(pipe, X, y, cv=kf5, scoring='neg_mean_squared_error',
                             return_train_score=False)
    mse5 = -np.mean(scores5['test_score'])
   
    # 10-fold CV
    pipe = make_pipeline(PolynomialFeatures(degree=deg, include_bias=False),
                         SMWrapper())
    scores10 = cross_validate(pipe, X, y, cv=kf10, scoring='neg_mean_squared_error',
                              return_train_score=False)
    mse10 = -np.mean(scores10['test_score'])
   
    # store rounded results
    results[deg] = {
        'mse_5fold': np.round(mse5, 3),
        'mse_10fold': np.round(mse10, 3),
        'diff_5_minus_10': np.round(mse5 - mse10, 3)
    }

# print results
for deg in degrees:
    r = results[deg]
    print(f"Degree {deg}: 5-fold MSE = {r['mse_5fold']:.3f}, "
          f"10-fold MSE = {r['mse_10fold']:.3f}, "
          f"Difference (5-fold - 10-fold) = {r['diff_5_minus_10']:.3f}")

Loaded College.csv: (777, 18)
Columns: ['Private', 'Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni', 'Expend', 'Grad.Rate']
Degree 1: 5-fold MSE = 9377937.163, 10-fold MSE = 9315542.590, Difference (5-fold - 10-fold) = 62394.573
Degree 2: 5-fold MSE = 9345970.983, 10-fold MSE = 9281464.281, Difference (5-fold - 10-fold) = 64506.702
Degree 3: 5-fold MSE = 9385687.233, 10-fold MSE = 9293767.859, Difference (5-fold - 10-fold) = 91919.374


# 4. Bootstrap


In [17]:
import numpy as np
import pandas as pd

# 1) Load data
path = '/Users/anhnguyen/Documents/GitHub/2025AAE722_AnhNguyen/2025 AAE722 Anh Nguyen Submission/ALL CSV FILES - 2nd Edition/Default.csv'
df = pd.read_csv(path)

# Keep only needed columns and drop any missing values
df = df[['balance', 'income']].dropna()

# 2) Function to compute the correlation coefficient
def corr_balance_income(frame: pd.DataFrame) -> float:
    return frame['balance'].corr(frame['income'])

# 3) Bootstrap resampling
B = 2000
rs = np.random.RandomState(456)   # random_state=456
n = len(df)

boot_corrs = np.empty(B, dtype=float)
for b in range(B):
    idx = rs.randint(0, n, size=n)   # sample with replacement
    sample = df.iloc[idx]
    boot_corrs[b] = corr_balance_income(sample)

# 4) Bootstrap standard error and actual correlation
boot_se = boot_corrs.std(ddof=1)
actual_corr = corr_balance_income(df)

# 5) Round to 4 decimals and print
boot_se_rounded = np.round(boot_se, 4)
actual_corr_rounded = np.round(actual_corr, 4)

print("Bootstrap SE (corr(balance, income)):", boot_se_rounded)
print("Actual correlation (full data):", actual_corr_rounded)

Bootstrap SE (corr(balance, income)): 0.0099
Actual correlation (full data): -0.1522


# 5. Traditional Standard Errors

In [18]:
# ---------------------------------------------------------------
# Wage.csv — Bootstrap vs. OLS SEs for wage ~ age + age^2
# B = 1500, seed = 789
# ---------------------------------------------------------------

import numpy as np
import pandas as pd
import statsmodels.api as sm

# 1) Load data
path = '/Users/anhnguyen/Documents/GitHub/2025AAE722_AnhNguyen/2025 AAE722 Anh Nguyen Submission/ALL CSV FILES - 2nd Edition/Wage.csv'
df = pd.read_csv(path)

# Keep only needed columns; drop missing
df = df[['wage', 'age']].dropna().copy()

# 2) Fit linear model: wage ~ age (for completeness)
X_lin = sm.add_constant(df[['age']])
y = df['wage'].astype(float)
lin_mod = sm.OLS(y, X_lin).fit()

# Optional: view traditional OLS SEs in the printed summary
# print(lin_mod.summary())

# 3) Fit quadratic model: wage ~ age + age^2
df['age2'] = df['age'] ** 2
X_quad = sm.add_constant(df[['age', 'age2']])
quad_mod = sm.OLS(y, X_quad).fit()

# Optional: OLS summary (contains the "traditional" SEs)
# print(quad_mod.summary())

# 4) Bootstrap SEs for quadratic model coefficients
B = 1500
rng = np.random.default_rng(789)  # seed=789
n = len(df)

boot_params = np.empty((B, 3))  # columns: Intercept, age, age^2

for b in range(B):
    idx = rng.integers(0, n, size=n)  # sample rows with replacement
    sample = df.iloc[idx]
    Xb = sm.add_constant(sample[['age', 'age2']])
    yb = sample['wage']
    # Fit OLS on bootstrap sample
    res_b = sm.OLS(yb, Xb).fit()
    boot_params[b, :] = res_b.params.values  # [const, age, age2]

# Bootstrap standard errors (sample std with ddof=1)
boot_se = boot_params.std(axis=0, ddof=1)

# 5) Traditional OLS SEs from the quadratic model
ols_se = quad_mod.bse.values  # [const, age, age2]

# 6) Ratios: bootstrap SE / OLS SE, rounded to 3 decimals
ratios = np.round(boot_se / ols_se, 3)

# 7) Report (quadratic model only)
labels = ['Intercept', 'age', 'age^2']
result = pd.Series(ratios, index=labels, name='Bootstrap SE / OLS SE (quad)')
print(result)

# If you also want them as a numpy array in the same order:
print("Ratios array (Intercept, age, age^2):", ratios)


Intercept    0.750
age          0.808
age^2        0.841
Name: Bootstrap SE / OLS SE (quad), dtype: float64
Ratios array (Intercept, age, age^2): [0.75  0.808 0.841]
